In [2]:
import sys
from pathlib import Path
from typing import List, Dict

from loguru import logger
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, mean

In [3]:
CONF_LOG_PREFIX = 'CONFLOG'
FLST_LOG_PREFIX = 'FLSTLOG'
GEO_LOG_PREFIX = 'GEOLOG'
LOS_LOG_PREFIX = 'LOSLOG'
REG_LOG_PREFIX = 'REGLOG'
LOADING_PATH = '/mnt/shared/repos/metropolis/M2_data_analysis_platform/output'
DATAFRAMES_NAMES = [CONF_LOG_PREFIX, FLST_LOG_PREFIX, GEO_LOG_PREFIX, LOS_LOG_PREFIX, REG_LOG_PREFIX]

Give access to the constants that defines

In [4]:
sys.path.append(str(Path(Path().absolute().parent, 'platform_code')))
from schemas.tables_attributes import *

In [5]:
def load_dataframes(files_names: List[str], loading_path: str, spark: SparkSession) -> Dict[str, DataFrame]:
    """ Loads the dataframes which macht the file names passed by arguments.
    The method read from the config the path were to read the files, which
    matches the folder where the files are saved in `save_dataframes_dict()`.

    :param files_names: list of the names of the files.
    :param loading_path: path were the files are saved.
    :param spark: spark session.
    :return: dictionary with the dataframes loaded from the files, with the
     file name as key.
    """
    dataframes = dict()

    for file_name in files_names:
        file_path = Path(loading_path, f'{file_name.lower()}.parquet')
        logger.info('Loading dataframe from `{}`.', file_path)
        df = spark.read.parquet(str(file_path))
        dataframes[file_name] = df

    return dataframes

In [6]:
spark = SparkSession.builder.appName('Notebook').getOrCreate()

22/03/24 16:37:08 WARN Utils: Your hostname, GRILUX-DEV resolves to a loopback address: 127.0.1.1; using 192.168.2.150 instead (on interface wlp3s0)
22/03/24 16:37:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/24 16:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
input_dataframes = load_dataframes(DATAFRAMES_NAMES, LOADING_PATH, spark)

2022-03-24 16:37:23.872 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `/mnt/shared/repos/metropolis/M2_data_analysis_platform/output/conflog.parquet`.
2022-03-24 16:37:29.531 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `/mnt/shared/repos/metropolis/M2_data_analysis_platform/output/flstlog.parquet`.
2022-03-24 16:37:29.782 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `/mnt/shared/repos/metropolis/M2_data_analysis_platform/output/geolog.parquet`.
2022-03-24 16:37:30.024 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `/mnt/shared/repos/metropolis/M2_data_analysis_platform/output/loslog.parquet`.
2022-03-24 16:37:30.241 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `/mnt/shared/repos/metropolis/M2_data_analysis_platform/output/reglog.parquet`.


For this metrics we are going to work with the combined FLST log and the flight intentions

# CAP-1: Average demand delay
Although a flight intention delay represents direct measure of the system efficiency, it can also be used as a proxy for the lack of the system capacity as previously explained. The more flights are delayed, the bigger the capacity problem is. Hence, it can be used for the relative comparison of the concepts

Average demand delay is computed as the arithmetic mean of the delays of all flight intentions in an scenario.

In [ ]:
dataframe = input_dataframes[FLST_LOG_PREFIX]

In [16]:
result = dataframe\
    .select(SCENARIO_NAME, BASELINE_ARRIVAL_TIME, DEL_TIME)\
    .groupby(SCENARIO_NAME)\
    .agg(mean(col(DEL_TIME) - col(BASELINE_ARRIVAL_TIME)).alias(CAP1))

In [17]:
result.show()

+------------------+------------------+
|          Scenario|              CAP1|
+------------------+------------------+
|1_very_low_40_8_R2|174.88071918752135|
|3_very_low_40_8_R2|174.88071918752135|
|1_very_low_40_8_W1|174.88071918752135|
|3_very_low_40_8_W1|174.88071918752135|
|2_very_low_40_8_W1|174.88071918752135|
|2_very_low_40_8_R2|174.88071918752135|
+------------------+------------------+



# CAP-2: Average number of intrusions
Is a ratio of the total number of intrusions with respect of the number of flight intention in the scenario.

In [18]:
los_log_df = input_dataframes[LOS_LOG_PREFIX]
flst_log_df = input_dataframes[FLST_LOG_PREFIX]

In [24]:
saf_2_result = los_log_df\
    .groupBy(SCENARIO_NAME)\
    .count()\
    .select(SCENARIO_NAME, col('count').alias(SAF2))

In [25]:
saf_2_result.show()

+------------------+----+
|          Scenario|SAF2|
+------------------+----+
|1_very_low_40_8_W1| 785|
|2_very_low_40_8_W1| 785|
|1_very_low_40_8_R2| 785|
|3_very_low_40_8_W1| 785|
|3_very_low_40_8_R2| 785|
|2_very_low_40_8_R2| 785|
+------------------+----+



In [26]:
number_of_flights = flst_log_df\
    .groupby(SCENARIO_NAME)\
    .count()\
    .select([SCENARIO_NAME, col('count').alias('num_flights')])

In [27]:
number_of_flights.show()

+------------------+-----------+
|          Scenario|num_flights|
+------------------+-----------+
|1_very_low_40_8_R2|       3760|
|3_very_low_40_8_R2|       3760|
|1_very_low_40_8_W1|       3760|
|3_very_low_40_8_W1|       3760|
|2_very_low_40_8_W1|       3760|
|2_very_low_40_8_R2|       3760|
+------------------+-----------+



In [30]:
result = saf_2_result.join(number_of_flights, on=SCENARIO_NAME)\
    .withColumn(CAP2, col(SAF2) / col('num_flights'))\
    .select(SCENARIO_NAME, CAP2)

In [31]:
result.show()

+------------------+-------------------+
|          Scenario|               CAP2|
+------------------+-------------------+
|1_very_low_40_8_R2|0.20877659574468085|
|3_very_low_40_8_R2|0.20877659574468085|
|1_very_low_40_8_W1|0.20877659574468085|
|3_very_low_40_8_W1|0.20877659574468085|
|2_very_low_40_8_W1|0.20877659574468085|
|2_very_low_40_8_R2|0.20877659574468085|
+------------------+-------------------+



# CAP-3: Additional demand delay
Calculates the magnitude of delay increase (CAP-1) due to the fact of the existence of rogue aircraft.

# CAP-4: Additional number of intrusions
Calculates the degradation produced in the intrusion safety indicator when rogue aircraft are introduced.